In [10]:
from fasta import FASTQ
from Bio import AlignIO
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

In [11]:
# Parameters for input and output
min_seq_len_to_report = 3
skip_rows = 1
set_destinated_protein_id_for_chimerax = "8VYN_1|Chains"
pdb_offset = 1
# List for plotting in ChimeraX for conservative regions
color_list = ["pink", "orange", "yellow", "white", "purple", "brown", "cyan",\
              "lime", "gray", "dark slate blue", "turquoise", "gold", "tan", "chocolate", "tomato", "violet",
              "indigo", "navy", "deep pink", "chartreuse", "spring green", "goldenrod", "black", "sky blue"]

color_list = ["yellow"] # Conserved
chain_color = "dark grey"  # Non-Conserved

# Dimer, Trimer? or even more?
chain = "A, B, C"

In [12]:
# Do part 1, run only at first time to do alignment.
try: 
    alignment = AlignIO.read("2_aligned.aln-clustal_num", "clustal")
    gen_align = False
except:
    gen_align = True

if gen_align == True:
    !clustalo -i 1_sequences.fasta --outfmt=clu --wrap=2048 --outfile=2_aligned.aln-clustal_num --force --threads=16;

In [13]:
# Read aligned data, missing part show with "-"
pdb_offset = pdb_offset - 1
alignment = AlignIO.read("2_aligned.aln-clustal_num", "clustal")
pre_seq_offset_in_clustal_file = 16 # Could be 16, 17, 20, 24, you are right when Length of sequence = After cut

try:
    shutil.copy("2_aligned.aln-clustal_num", "3_aligned.aln-clustal_num.txt")
    f = open("3_aligned.aln-clustal_num.txt", "r")
except:
    print("Failed to open file.")

found_offset = False
loop_count = 0
i = 0

for x in f:
    i = i + 1

    if len(set_destinated_protein_id_for_chimerax) > 1:
        destinated_protein_id_for_chimerax = set_destinated_protein_id_for_chimerax
        break

    if i > skip_rows and np.size([*x]) > 1:
        destinated_protein_id_for_chimerax = "".join([*x][0:x.find(".") + 2])
        break

print("First Protein is:", destinated_protein_id_for_chimerax)

First Protein is: 8VYN_1|Chains


In [14]:
# Read aligned data, missing part show with "-"
alignment = AlignIO.read("2_aligned.aln-clustal_num", "clustal")
pre_seq_offset_in_clustal_file = 16 # Could be 16, 17, 20, 24, you are right when Length of sequence = After cut
f = open("3_aligned.aln-clustal_num.txt", "r")

found_offset = False
loop_count = 0

while found_offset == False:
    conservative_seq = ""
    i = 0
    loop_count += 1
    alignment = AlignIO.read("2_aligned.aln-clustal_num", "clustal")
    f = open("3_aligned.aln-clustal_num.txt", "r")

    for x in f:
        i = i + 1
        if i >= skip_rows:
            if x[0:pre_seq_offset_in_clustal_file] == " " * pre_seq_offset_in_clustal_file:
                conservative_seq += x[pre_seq_offset_in_clustal_file:]
                #print(conservative_seq)
        
    conservative_seq = conservative_seq.replace('\n',"")
    conservative_seq = [*conservative_seq]

    for record in alignment:
        if record.id == destinated_protein_id_for_chimerax:
            #print(record.id)
            sequence = str(record.seq)
            seq_replaced = [*sequence.replace("-", "")]
            #print(sequence)
            sequence = np.array([*sequence])
            rm_index = np.where(sequence == "-")[0]
            #print("Length of sequence:", len(seq_replaced))

    rm_index = rm_index[::-1]
    #print("Before cut", len(conservative_seq))

    for j in rm_index:
        del conservative_seq[int(j)]

    #print("After cut", len(conservative_seq))

    seq_len = len(seq_replaced)
    cut_end = len(conservative_seq)

    if cut_end == 0 or seq_len == 0:
        pre_seq_offset_in_clustal_file = pre_seq_offset_in_clustal_file - 1

    
    elif seq_len == cut_end:
        found_offset = True
        print("Found Offset:", pre_seq_offset_in_clustal_file)
        print("Ref_seq_Id", destinated_protein_id_for_chimerax)
        print("Sequence", seq_replaced)
        print("Length of sequence:", len(seq_replaced))
        print("Before cut", len(conservative_seq))
        print("After cut", len(conservative_seq))

    elif seq_len < cut_end:
        pre_seq_offset_in_clustal_file = pre_seq_offset_in_clustal_file + 1

    if loop_count > 32:
        raise("FileExistsError", "Alignment file is wrong or corrpted")

Found Offset: 19
Ref_seq_Id 8VYN_1|Chains
Sequence ['M', 'E', 'S', 'R', 'I', 'W', 'C', 'L', 'V', 'V', 'C', 'V', 'N', 'L', 'C', 'I', 'V', 'C', 'L', 'G', 'A', 'A', 'V', 'S', 'S', 'S', 'S', 'T', 'R', 'G', 'T', 'S', 'A', 'T', 'H', 'S', 'H', 'H', 'S', 'S', 'H', 'T', 'T', 'S', 'A', 'A', 'H', 'S', 'R', 'S', 'G', 'S', 'V', 'S', 'Q', 'R', 'V', 'T', 'S', 'S', 'Q', 'T', 'V', 'S', 'H', 'G', 'V', 'N', 'E', 'T', 'I', 'Y', 'N', 'T', 'T', 'L', 'K', 'Y', 'G', 'D', 'V', 'V', 'G', 'V', 'N', 'T', 'T', 'K', 'Y', 'P', 'Y', 'R', 'V', 'C', 'S', 'M', 'A', 'Q', 'G', 'L', 'D', 'L', 'I', 'R', 'F', 'E', 'R', 'N', 'I', 'V', 'C', 'T', 'S', 'M', 'K', 'P', 'I', 'N', 'E', 'D', 'L', 'D', 'E', 'G', 'I', 'M', 'V', 'V', 'Y', 'K', 'R', 'N', 'I', 'C', 'A', 'H', 'T', 'F', 'K', 'V', 'R', 'V', 'Y', 'Q', 'K', 'V', 'L', 'T', 'F', 'R', 'R', 'S', 'Y', 'A', 'Y', 'I', 'H', 'T', 'T', 'Y', 'L', 'L', 'G', 'S', 'N', 'T', 'E', 'Y', 'V', 'A', 'P', 'P', 'M', 'W', 'E', 'I', 'H', 'H', 'I', 'N', 'S', 'H', 'S', 'Q', 'C', 'Y', 'S', 'S', 'Y', 'S'

In [15]:
# Debug to check alignment and cutting is correct
for i in range(np.size(seq_replaced)):
    print("Index", i + 1 + pdb_offset, seq_replaced[i], conservative_seq[i])

Index 1 M  
Index 2 E  
Index 3 S  
Index 4 R  
Index 5 I  
Index 6 W  
Index 7 C  
Index 8 L  
Index 9 V  
Index 10 V  
Index 11 C  
Index 12 V  
Index 13 N  
Index 14 L  
Index 15 C  
Index 16 I  
Index 17 V  
Index 18 C  
Index 19 L  
Index 20 G  
Index 21 A  
Index 22 A  
Index 23 V  
Index 24 S  
Index 25 S  
Index 26 S  
Index 27 S  
Index 28 T  
Index 29 R  
Index 30 G  
Index 31 T  
Index 32 S  
Index 33 A  
Index 34 T  
Index 35 H  
Index 36 S  
Index 37 H  
Index 38 H  
Index 39 S  
Index 40 S  
Index 41 H  
Index 42 T  
Index 43 T  
Index 44 S  
Index 45 A  
Index 46 A  
Index 47 H  
Index 48 S  
Index 49 R  
Index 50 S  
Index 51 G  
Index 52 S  
Index 53 V  
Index 54 S  
Index 55 Q  
Index 56 R  
Index 57 V  
Index 58 T  
Index 59 S  
Index 60 S  
Index 61 Q  
Index 62 T  
Index 63 V  
Index 64 S  
Index 65 H  
Index 66 G  
Index 67 V  
Index 68 N  
Index 69 E  
Index 70 T  
Index 71 I  
Index 72 Y  
Index 73 N  
Index 74 T  
Index 75 T  
Index 76 L  
Index 77 K  
Index 78

In [16]:
# Report in text
current_step = []
tot_size = 0

for i in range(np.size(conservative_seq)):
    if conservative_seq[i] == "*":
        current_step.append(i + 1)

    elif conservative_seq[i] == ":":
        current_step.append(i + 1)

    elif np.size(current_step) >= min_seq_len_to_report:
        print("Found conservation region at amino position {0} - {1}, with length {2}".format(current_step[0] + pdb_offset, current_step[-1] + pdb_offset, np.size(current_step)))
        tot_size = tot_size + np.size(current_step)
        current_step = []

    elif np.size(current_step) < min_seq_len_to_report:
        tot_size = tot_size + np.size(current_step)
        current_step = []
    
    else:
        pass

try:
    print("Found conservation region at amino position {0} - {1}, with length {2}".format(current_step[0] + pdb_offset, current_step[-1] + pdb_offset, np.size(current_step)))
except:
    pass

print("Score Ratio", np.round(tot_size / seq_len, 2))

Score Ratio 0.0


In [17]:
# Report in Command
current_step = []
print("select /{1}; color sel {0}".format(chain_color, chain))

for i in range(np.size(conservative_seq)):
    if conservative_seq[i] == "*":
        current_step.append(i + 1)

    elif conservative_seq[i] == ":":
        current_step.append(i + 1)

    elif conservative_seq[i] == ".":
        current_step.append(i + 1)

    elif np.size(current_step) >= min_seq_len_to_report:
        print("select /{0}:{1}-{2}; color sel {3}".format(chain, current_step[0] + pdb_offset, current_step[-1] + pdb_offset, color_list[i%np.size(color_list)]))
        current_step = []

    elif np.size(current_step) < min_seq_len_to_report:
        current_step = []
    
    else:
        pass

try:
    print("select /{0}:{1}-{2}; color sel {3}".format(chain, current_step[0] + pdb_offset, current_step[-1] + pdb_offset, color_list[i%np.size(color_list)]))
except:
    pass

print('preset "{0}" "{1}"'.format("overall look", "publication 1 (silhouettes)"))
print("save 5_chimera.cxs")

select /A, B, C; color sel dark grey
preset "overall look" "publication 1 (silhouettes)"
save 5_chimera.cxs


In [18]:
try:
    !chimerax;
except:
    pass

NOTE: available bundle cache has not been initialized yet
